In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, recall_score,f1_score,make_scorer
from sklearn.model_selection import GridSearchCV
from imblearn.under_sampling import RandomUnderSampler

In [48]:
data = pd.read_csv("serieb_modelagem2.csv")

In [7]:
data.head()

,idteamgame,idopponentgame,tournament,venue,result,points,gf,ga,possession,gdiff,shots,p_shotsontarget,goals_shots,penalty_goals,p_penaltyconverted,shotsontarget_against,p_saves,cleansheets,penalty_attempted_against,p_penalty_saved,cardscore,foulsdiff,offsides,crosses,interceptions,tackles_won,own_goals,opp_points,opp_gf,opp_ga,opp_possession,opp_gdiff,opp_shots,opp_p_shotsontarget,opp_goals_shots,opp_penalty_goals,opp_p_penaltyconverted,opp_shotsontarget_against,opp_p_saves,opp_cleansheets,opp_penalty_attempted_against,opp_p_penalty_saved,opp_cardscore,opp_foulsdiff,opp_offsides,opp_crosses,opp_interceptions,opp_tackles_won,opp_own_goals
0,ABC_12.0_2023,Chapecoense_12.0_2023,2023,Home,D,4.0,0.4,1.2,0.454,-4.0,9.4,0.2462,0.022,0.0,0.0,5.4,0.7858,2.0,0.0,0.0,2.6,2.2,1.8,13.0,12.6,12.0,0.0,1.0,0.6,1.8,0.516,-6.0,14.6,0.2728,0.028,0.0,0.0,5.6,0.6206,0.0,1.0,0.2,4.4,2.6,2.4,26.2,7.8,8.2,1.0
1,ABC_15.0_2023,Avaí_15.0_2023,2023,Away,W,3.0,0.8,1.2,0.446,-2.0,10.8,0.2306,0.050,2.0,0.4,6.0,0.7834,1.0,0.0,0.0,4.0,1.8,1.6,17.4,11.8,13.2,0.0,3.0,0.8,1.2,0.440,-2.0,14.0,0.2522,0.044,1.0,0.2,3.0,0.6808,2.0,1.0,0.2,5.8,-5.2,0.8,19.4,12.2,8.6,0.0
2,ABC_16.0_2023,Criciúma_16.0_2023,2023,Home,L,6.0,1.2,1.0,0.414,1.0,11.0,0.3084,0.090,2.0,0.4,5.2,0.8168,2.0,0.0,0.0,4.0,0.2,1.4,15.4,10.2,12.0,0.0,10.0,1.2,0.6,0.466,3.0,12.4,0.3520,0.112,0.0,0.0,2.8,0.7668,2.0,0.0,0.0,2.8,-1.2,1.2,15.4,10.6,12.0,0.0
3,ABC_17.0_2023,Juventude_17.0_2023,2023,Away,D,5.0,1.2,1.2,0.462,0.0,13.4,0.3964,0.090,2.0,0.4,4.2,0.8168,1.0,1.0,0.0,4.0,-0.6,1.4,23.2,8.8,10.4,0.0,7.0,1.0,1.4,0.520,-2.0,13.6,0.2792,0.082,0.0,0.0,4.6,0.7252,1.0,0.0,0.0,4.2,-0.6,1.8,23.0,8.6,7.8,0.0
4,ABC_18.0_2023,Guarani_18.0_2023,2023,Home,L,5.0,1.2,1.2,0.456,0.0,11.6,0.5630,0.156,1.0,0.2,4.2,0.8834,1.0,2.0,0.0,5.4,-3.8,1.8,21.6,9.8,9.8,0.0,10.0,1.4,0.8,0.436,3.0,9.8,0.3600,0.164,1.0,0.2,3.8,0.7584,1.0,0.0,0.0,4.0,0.2,1.8,13.4,7.2,9.6,0.0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2070 entries, 0 to 2069
Data columns (total 49 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   idteamgame                     2070 non-null   object 
 1   idopponentgame                 2070 non-null   object 
 2   tournament                     2070 non-null   int64  
 3   venue                          2070 non-null   object 
 4   result                         2070 non-null   object 
 5   points                         2070 non-null   float64
 6   gf                             2070 non-null   float64
 7   ga                             2070 non-null   float64
 8   possession                     2070 non-null   float64
 9   gdiff                          2070 non-null   float64
 10  shots                          2070 non-null   float64
 11  p_shotsontarget                2070 non-null   float64
 12  goals_shots                    2070 non-null   f

In [49]:
data['venue'] = data['venue'].replace(['Home','Away'],[1,0])
data = data.drop(columns=['idteamgame','idopponentgame','tournament'])

### 1. Tipo 1 - Todas as variaveis

In [50]:
X = data.drop(columns=['result'])
y = data['result']

In [51]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=325)

X_test,X_val,y_test,y_val = train_test_split(X_test,y_test,test_size=0.5, random_state=3255)

In [52]:
# Dados
print("Total")
print(len(X.index))
print(len(y.index))

# Treino e Teste (geral)
print("Treino")
print(len(X_train.index))
print(len(y_train.index))

print("Teste")
print(len(X_test.index))
print(len(y_test.index))

print("Validação")
print(len(X_val.index))
print(len(y_val.index))

Total
2070
2070
Treino
1449
1449
Teste
310
310
Validação
311
311


In [53]:
model_1 = RandomForestClassifier()
model_1.fit(X_train,y_train)

RandomForestClassifier()

In [54]:
predictions_1 = model_1.predict(X_test)

In [55]:
print("Confusion Matrix")
print(confusion_matrix(y_test,predictions_1))
print("Accuracy:",accuracy_score(y_test,predictions_1))
print("Precision:",precision_score(y_test,predictions_1,average='weighted'))
print("Sensitivity:",recall_score(y_test,predictions_1,average='weighted')) #Sensitivity
print("Specifity:",recall_score(y_test,predictions_1,pos_label=0,average='weighted')) #Specifity
print("F1-Score:",f1_score(y_test,predictions_1,average='weighted'))

Confusion Matrix
[[26 32 33]
 [25 49 33]
 [20 35 57]]
Accuracy: 0.4258064516129032
Precision: 0.42072470499493886
Sensitivity: 0.4258064516129032
Specifity: 0.4258064516129032
F1-Score: 0.4211748805209373


In [62]:
predictions_g2 = model_1.predict(X_val)

In [57]:
print("Confusion Matrix")
print(confusion_matrix(y_val,predictions_2))
print("Accuracy:",accuracy_score(y_val,predictions_2))
print("Precision:",precision_score(y_val,predictions_2,average='weighted'))
print("Sensitivity:",recall_score(y_val,predictions_2,average='weighted')) #Sensitivity
print("Specifity:",recall_score(y_val,predictions_2,pos_label=0,average='weighted')) #Specifity
print("F1-Score:",f1_score(y_val,predictions_2,average='weighted'))

Confusion Matrix
[[23 33 27]
 [31 46 36]
 [29 34 52]]
Accuracy: 0.3890675241157556
Precision: 0.3890675241157556
Sensitivity: 0.3890675241157556
Specifity: 0.3890675241157556
F1-Score: 0.3890675241157556


In [58]:
prob_res = model_1.predict_proba(X_val)

In [60]:
predictions_2[0:5]

array(['D', 'W', 'D', 'W', 'D'], dtype=object)

In [61]:
prob_res[0:5]

array([[0.4 , 0.35, 0.25],
       [0.28, 0.3 , 0.42],
       [0.37, 0.27, 0.36],
       [0.22, 0.38, 0.4 ],
       [0.42, 0.22, 0.36]])

### 2. Tipo 2 - subtrair variáveis

In [4]:
data_2 = data

In [5]:
data_2['points'] = data_2['points'] - data_2['opp_points']
data_2['possession'] = data_2['possession'] - data_2['opp_possession']
data_2['gdiff'] = data_2['gdiff'] - data_2['opp_gdiff']
data_2['shots'] = data_2['shots'] - data_2['opp_shots']
data_2['p_shotsontarget'] = data_2['p_shotsontarget'] - data_2['opp_p_shotsontarget']
data_2['goals_shots'] = data_2['goals_shots'] - data_2['opp_goals_shots']
data_2['penalty_goals'] = data_2['penalty_goals'] - data_2['opp_penalty_goals']
data_2['p_penaltyconverted'] = data_2['p_penaltyconverted'] - data_2['opp_p_penaltyconverted']
data_2['shotsontarget_against'] = data_2['shotsontarget_against'] - data_2['opp_shotsontarget_against']
data_2['p_saves'] = data_2['p_saves'] - data_2['opp_p_saves']
data_2['cleansheets'] = data_2['cleansheets'] - data_2['opp_cleansheets']
data_2['penalty_attempted_against'] = data_2['penalty_attempted_against'] - data_2['opp_penalty_attempted_against']
data_2['p_penalty_saved'] = data_2['p_penalty_saved'] - data_2['opp_p_penalty_saved']
data_2['cardscore'] = data_2['cardscore'] - data_2['opp_cardscore']
data_2['foulsdiff'] = data_2['foulsdiff'] - data_2['opp_foulsdiff']
data_2['offsides'] = data_2['offsides'] - data_2['opp_offsides']
data_2['crosses'] = data_2['crosses'] - data_2['opp_crosses']
data_2['interceptions'] = data_2['interceptions'] - data_2['opp_interceptions']
data_2['tackles_won'] = data_2['tackles_won'] - data_2['opp_tackles_won']
data_2['own_goals'] = data_2['own_goals'] - data_2['opp_own_goals']

In [6]:
data_2 = data_2.iloc[:,0:22]

In [7]:
X = data_2.drop(columns=['result'])
y = data_2['result']

In [71]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=325)

X_test,X_val,y_test,y_val = train_test_split(X_test,y_test,test_size=0.5, random_state=3255)

In [72]:
model_1 = RandomForestClassifier(criterion='log_loss', max_depth=4, max_features=None)
model_1.fit(X_train,y_train)

RandomForestClassifier(criterion='log_loss', max_depth=4, max_features=None)

In [73]:
predictions_1 = model_1.predict(X_test)

In [74]:
print("Confusion Matrix")
print(confusion_matrix(y_test,predictions_1))
print("Accuracy:",accuracy_score(y_test,predictions_1))
print("Precision:",precision_score(y_test,predictions_1,average='weighted'))
print("Sensitivity:",recall_score(y_test,predictions_1,average='weighted')) #Sensitivity
print("Specifity:",recall_score(y_test,predictions_1,pos_label=0,average='weighted')) #Specifity
print("F1-Score:",f1_score(y_test,predictions_1,average='weighted'))

Confusion Matrix
[[ 7 33 37]
 [ 5 63 32]
 [ 1 51 81]]
Accuracy: 0.4870967741935484
Precision: 0.5036731655441333
Sensitivity: 0.4870967741935484
Specifity: 0.4870967741935484
F1-Score: 0.4487877391778716


In [61]:
predictions_2 = model_1.predict(X_val)

In [62]:
print("Confusion Matrix")
print(confusion_matrix(y_val,predictions_2))
print("Accuracy:",accuracy_score(y_val,predictions_2))
print("Precision:",precision_score(y_val,predictions_2,average='weighted'))
print("Sensitivity:",recall_score(y_val,predictions_2,average='weighted')) #Sensitivity
print("Specifity:",recall_score(y_val,predictions_2,pos_label=0,average='weighted')) #Specifity
print("F1-Score:",f1_score(y_val,predictions_2,average='weighted'))

Confusion Matrix
[[27 38 32]
 [19 55 37]
 [27 39 37]]
Accuracy: 0.38263665594855306
Precision: 0.37967698558664426
Sensitivity: 0.38263665594855306
Specifity: 0.38263665594855306
F1-Score: 0.37790217944585325


### 3 - Subtraindo valores e considerando apenas o time da casa

In [79]:
data_3 = data_2[data_2['venue'] == 1]
data_3 = data_3.drop(columns=['venue'])

In [80]:
data_3.head()

,result,points,possession,gdiff,shots,p_shotsontarget,goals_shots,penalty_goals,p_penaltyconverted,shotsontarget_against,p_saves,cleansheets,penalty_attempted_against,p_penalty_saved,cardscore,foulsdiff,offsides,crosses,interceptions,tackles_won,own_goals
3,D,-22.0,-1.278,-1.0,-41.0,-0.6852,-0.236,1.0,0.2,-6.8,-1.7200,-5.0,0.0,0.0,-10.0,33.0,-8.4,-61.8,-24.8,-45.8,0.0
5,W,-24.0,-1.080,1.0,-17.2,-1.8088,-0.564,-4.0,-0.8,-17.0,-2.1686,-2.0,0.0,0.0,-13.6,14.0,-4.2,-33.0,-35.8,-33.6,0.0
7,D,-44.0,-1.332,-23.0,-34.2,-1.0434,-0.490,0.0,0.0,-12.0,-2.9670,-9.0,0.0,0.0,-8.0,15.8,-6.8,-56.8,-35.6,-37.4,0.0
9,W,-4.0,-1.684,19.0,-53.6,-0.6432,-0.082,-4.0,-0.4,-8.2,-1.7622,1.0,-8.0,0.0,-14.0,3.6,-2.6,-65.6,-17.6,-44.4,0.0
11,W,-10.0,-1.456,10.0,-33.0,-0.6348,-0.156,1.0,0.2,-15.6,-3.1200,-11.0,1.0,0.2,-11.2,-6.8,-4.0,-68.4,-37.4,-28.0,0.0


In [81]:
X = data_3.drop(columns=['result'])
y = data_3['result']

In [82]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=325)

X_test,X_val,y_test,y_val = train_test_split(X_test,y_test,test_size=0.5, random_state=3255)

In [96]:
model_1 = RandomForestClassifier()
model_1.fit(X_train,y_train)

RandomForestClassifier()

In [97]:
predictions_1 = model_1.predict(X_test)

In [98]:
print("Confusion Matrix")
print(confusion_matrix(y_test,predictions_1))
print("Accuracy:",accuracy_score(y_test,predictions_1))
print("Precision:",precision_score(y_test,predictions_1,average='weighted'))
print("Sensitivity:",recall_score(y_test,predictions_1,average='weighted')) #Sensitivity
print("Specifity:",recall_score(y_test,predictions_1,pos_label=0,average='weighted')) #Specifity
print("F1-Score:",f1_score(y_test,predictions_1,average='weighted'))

Confusion Matrix
[[12  6 33]
 [ 9  3 28]
 [15  3 46]]
Accuracy: 0.3935483870967742
Precision: 0.351703346397347
Sensitivity: 0.3935483870967742
Specifity: 0.3935483870967742
F1-Score: 0.34269095916540443


In [99]:
predictions_2 = model_1.predict(X_val)

In [100]:
print("Confusion Matrix")
print(confusion_matrix(y_val,predictions_2))
print("Accuracy:",accuracy_score(y_val,predictions_2))
print("Precision:",precision_score(y_val,predictions_2,average='weighted'))
print("Sensitivity:",recall_score(y_val,predictions_2,average='weighted')) #Sensitivity
print("Specifity:",recall_score(y_val,predictions_2,pos_label=0,average='weighted')) #Specifity
print("F1-Score:",f1_score(y_val,predictions_2,average='weighted'))

Confusion Matrix
[[ 5  4 42]
 [ 9  5 28]
 [ 7  1 54]]
Accuracy: 0.4129032258064516
Precision: 0.38801843317972345
Sensitivity: 0.4129032258064516
Specifity: 0.4129032258064516
F1-Score: 0.330066170388751


### 4 - SUbstraindo e considerando vitoria/nao vitoria

In [8]:
data_2['result'] = data_2['result'].replace(['W','D','L'],[1,0,0])

In [10]:
X = data_2.drop(columns=['result'])
y = data_2['result']

In [11]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=325)

X_test,X_val,y_test,y_val = train_test_split(X_test,y_test,test_size=0.5, random_state=3255)

In [12]:
model_1 = RandomForestClassifier()
model_1.fit(X_train,y_train)

RandomForestClassifier()

In [13]:
predictions_1 = model_1.predict(X_test)

In [14]:
print("Confusion Matrix")
print(confusion_matrix(y_test,predictions_1))
print("Accuracy:",accuracy_score(y_test,predictions_1))
print("Precision:",precision_score(y_test,predictions_1,average='weighted'))
print("Sensitivity:",recall_score(y_test,predictions_1,average='weighted')) #Sensitivity
print("Specifity:",recall_score(y_test,predictions_1,pos_label=0,average='weighted')) #Specifity
print("F1-Score:",f1_score(y_test,predictions_1,average='weighted'))

Confusion Matrix
[[162  15]
 [111  22]]
Accuracy: 0.5935483870967742
Precision: 0.5939162842388649
Sensitivity: 0.5935483870967742
Specifity: 0.5935483870967742
F1-Score: 0.5221404174573056


In [15]:
predictions_2 = model_1.predict(X_val)

In [16]:
print("Confusion Matrix")
print(confusion_matrix(y_val,predictions_2))
print("Accuracy:",accuracy_score(y_val,predictions_2))
print("Precision:",precision_score(y_val,predictions_2,average='weighted'))
print("Sensitivity:",recall_score(y_val,predictions_2,average='weighted')) #Sensitivity
print("Specifity:",recall_score(y_val,predictions_2,pos_label=0,average='weighted')) #Specifity
print("F1-Score:",f1_score(y_val,predictions_2,average='weighted'))

Confusion Matrix
[[189  19]
 [ 88  15]]
Accuracy: 0.6559485530546624
Precision: 0.6024493700576101
Sensitivity: 0.6559485530546624
Specifity: 0.6559485530546624
F1-Score: 0.5937816229316174


### 5 - Tentando melhorar o modelo

In [93]:
# Randomized Search
param_dist = {"criterion": ['gini', 'entropy', 'log_loss'],
              "max_depth":[4,6,8,None],
              "max_features":['sqrt', 'log2', None]}

In [94]:
random_search = RandomizedSearchCV(estimator=model_1, param_distributions=param_dist,n_iter=40,cv=5)

In [65]:
random_search.fit(X,y)

KeyboardInterrupt: 

In [66]:
random_search.best_estimator_

RandomForestClassifier(criterion='log_loss', max_depth=4, max_features=None)

# Todas as Variáveis - Modelos Isolados (Vitória, Empate e Derrota)

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, recall_score,f1_score,make_scorer,classification_report
from sklearn.model_selection import GridSearchCV
from imblearn.under_sampling import RandomUnderSampler

In [4]:
data = pd.read_csv("serieb_modelagem2.csv")
data['venue'] = data['venue'].replace(['Home','Away'],[1,0])
data = data.drop(columns=['idteamgame','idopponentgame','tournament'])

data_1 = data
data_2 = data
data_3 = data
data_1['result'] = data_1['result'].replace(['W','D','L'],[1,0,0]) 
data_2['result'] = data_2['result'].replace(['W','D','L'],[0,1,0]) 
data_3['result'] = data_3['result'].replace(['W','D','L'],[0,0,1]) 

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2070 entries, 0 to 2069
Data columns (total 46 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   venue                          2070 non-null   int64  
 1   result                         2070 non-null   int64  
 2   points                         2070 non-null   float64
 3   gf                             2070 non-null   float64
 4   ga                             2070 non-null   float64
 5   possession                     2070 non-null   float64
 6   gdiff                          2070 non-null   float64
 7   shots                          2070 non-null   float64
 8   p_shotsontarget                2070 non-null   float64
 9   goals_shots                    2070 non-null   float64
 10  penalty_goals                  2070 non-null   float64
 11  p_penaltyconverted             2070 non-null   float64
 12  shotsontarget_against          2070 non-null   f

In [6]:
data.head(10)

,venue,result,points,gf,ga,possession,gdiff,shots,p_shotsontarget,goals_shots,penalty_goals,p_penaltyconverted,shotsontarget_against,p_saves,cleansheets,penalty_attempted_against,p_penalty_saved,cardscore,foulsdiff,offsides,crosses,interceptions,tackles_won,own_goals,opp_points,opp_gf,opp_ga,opp_possession,opp_gdiff,opp_shots,opp_p_shotsontarget,opp_goals_shots,opp_penalty_goals,opp_p_penaltyconverted,opp_shotsontarget_against,opp_p_saves,opp_cleansheets,opp_penalty_attempted_against,opp_p_penalty_saved,opp_cardscore,opp_foulsdiff,opp_offsides,opp_crosses,opp_interceptions,opp_tackles_won,opp_own_goals
0,1,0,4.0,0.4,1.2,0.454,-4.0,9.4,0.2462,0.022,0.0,0.0,5.4,0.7858,2.0,0.0,0.0,2.6,2.2,1.8,13.0,12.6,12.0,0.0,1.0,0.6,1.8,0.516,-6.0,14.6,0.2728,0.028,0.0,0.0,5.6,0.6206,0.0,1.0,0.2,4.4,2.6,2.4,26.2,7.8,8.2,1.0
1,0,1,3.0,0.8,1.2,0.446,-2.0,10.8,0.2306,0.050,2.0,0.4,6.0,0.7834,1.0,0.0,0.0,4.0,1.8,1.6,17.4,11.8,13.2,0.0,3.0,0.8,1.2,0.440,-2.0,14.0,0.2522,0.044,1.0,0.2,3.0,0.6808,2.0,1.0,0.2,5.8,-5.2,0.8,19.4,12.2,8.6,0.0
2,1,0,6.0,1.2,1.0,0.414,1.0,11.0,0.3084,0.090,2.0,0.4,5.2,0.8168,2.0,0.0,0.0,4.0,0.2,1.4,15.4,10.2,12.0,0.0,10.0,1.2,0.6,0.466,3.0,12.4,0.3520,0.112,0.0,0.0,2.8,0.7668,2.0,0.0,0.0,2.8,-1.2,1.2,15.4,10.6,12.0,0.0
3,0,0,5.0,1.2,1.2,0.462,0.0,13.4,0.3964,0.090,2.0,0.4,4.2,0.8168,1.0,1.0,0.0,4.0,-0.6,1.4,23.2,8.8,10.4,0.0,7.0,1.0,1.4,0.520,-2.0,13.6,0.2792,0.082,0.0,0.0,4.6,0.7252,1.0,0.0,0.0,4.2,-0.6,1.8,23.0,8.6,7.8,0.0
4,1,0,5.0,1.2,1.2,0.456,0.0,11.6,0.5630,0.156,1.0,0.2,4.2,0.8834,1.0,2.0,0.0,5.4,-3.8,1.8,21.6,9.8,9.8,0.0,10.0,1.4,0.8,0.436,3.0,9.8,0.3600,0.164,1.0,0.2,3.8,0.7584,1.0,0.0,0.0,4.0,0.2,1.8,13.4,7.2,9.6,0.0
5,1,0,5.0,0.8,0.8,0.446,0.0,10.0,0.4880,0.140,0.0,0.0,2.6,0.8600,1.0,2.0,0.0,4.2,-4.0,1.6,17.8,10.6,8.2,0.0,3.0,0.8,1.8,0.472,-5.0,10.0,0.2560,0.150,0.0,0.0,4.8,0.5416,1.0,1.0,0.0,4.0,-4.6,1.2,17.4,13.8,13.4,0.0
6,0,0,3.0,0.4,0.8,0.486,-2.0,11.4,0.4232,0.100,0.0,0.0,2.2,0.6600,2.0,2.0,0.0,4.0,-2.0,1.2,19.2,10.0,8.4,0.0,10.0,1.4,1.0,0.546,2.0,15.2,0.2634,0.072,1.0,0.2,2.2,0.5668,1.0,0.0,0.0,3.4,0.0,0.8,25.6,9.0,9.4,1.0
7,0,0,3.0,0.4,1.0,0.438,-3.0,8.2,0.4018,0.100,0.0,0.0,2.8,0.5600,2.0,1.0,0.0,4.2,-0.2,0.8,11.2,10.6,7.8,1.0,6.0,0.6,0.6,0.466,0.0,11.8,0.2878,0.024,1.0,0.2,3.6,0.9250,2.0,1.0,0.0,4.8,-6.2,1.2,16.8,11.0,10.0,0.0
8,1,0,2.0,0.2,1.0,0.458,-4.0,9.2,0.2768,0.034,0.0,0.0,2.8,0.4934,2.0,0.0,0.0,2.6,1.0,0.6,11.4,10.6,8.8,1.0,5.0,0.6,0.8,0.550,-1.0,15.6,0.2374,0.056,0.0,0.0,4.0,0.7714,1.0,0.0,0.0,3.4,-4.6,1.0,25.0,10.2,8.8,0.0
9,1,0,3.0,0.4,1.0,0.450,-3.0,10.2,0.2466,0.052,0.0,0.0,3.0,0.5268,2.0,0.0,0.0,3.0,3.8,0.6,13.0,10.2,8.8,1.0,8.0,0.8,0.8,0.518,0.0,11.0,0.3474,0.074,0.0,0.0,3.0,0.7266,2.0,2.0,0.0,3.2,0.2,1.4,18.2,6.6,11.8,0.0


In [7]:
rus = RandomUnderSampler(random_state=42)

## 1 - Vitória

In [8]:
X = data_1.drop(columns=['result'])
y = data_1['result']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y,random_state=325)
X_test,X_val,y_test,y_val = train_test_split(X_test,y_test,test_size=0.5,stratify=y_test, random_state=3255)

In [9]:
X_train, y_train = rus.fit_resample(X_train, y_train)
X_test, y_test = rus.fit_resample(X_test, y_test)
X_val, y_val = rus.fit_resample(X_val, y_val)

In [10]:
model_1 = RandomForestClassifier(max_features='log2')
model_1.fit(X_train,y_train)

RandomForestClassifier(max_features='log2')

In [11]:
predictions_1 = model_1.predict(X_test)

In [12]:
print("Confusion Matrix")
print(confusion_matrix(y_test,predictions_1))
print(f'Accuracy:{accuracy_score(y_test,predictions_1):.3f}')
print(f'Precision: {precision_score(y_test,predictions_1,average="weighted"):.3f}')
print(f'Sensitivity: {recall_score(y_test,predictions_1,average="weighted"):.3f}') #Sensitivity
print(f'Specifity: {recall_score(y_test,predictions_1,pos_label=0,average="weighted"):.3f}') #Specifity
print(f'F1-Score: {f1_score(y_test,predictions_1,average="weighted"):.3f}')
print(classification_report(y_test, predictions_1))

Confusion Matrix
[[68 41]
 [55 54]]
Accuracy:0.560
Precision: 0.561
Sensitivity: 0.560
Specifity: 0.560
F1-Score: 0.558
              precision    recall  f1-score   support

           0       0.55      0.62      0.59       109
           1       0.57      0.50      0.53       109

    accuracy                           0.56       218
   macro avg       0.56      0.56      0.56       218
weighted avg       0.56      0.56      0.56       218



In [13]:
predictions_2 = model_1.predict(X_val)

In [14]:
prob_vit = model_1.predict_proba(X_val)

In [15]:
print("Confusion Matrix")
print(confusion_matrix(y_val,predictions_2))
print("Accuracy:",accuracy_score(y_val,predictions_2))
print("Precision:",precision_score(y_val,predictions_2,average='weighted'))
print("Sensitivity:",recall_score(y_val,predictions_2,average='weighted')) #Sensitivity
print("Specifity:",recall_score(y_val,predictions_2,pos_label=0,average='weighted')) #Specifity
print("F1-Score:",f1_score(y_val,predictions_2,average='weighted'))
print(classification_report(y_val, predictions_2))

Confusion Matrix
[[59 51]
 [54 56]]
Accuracy: 0.5227272727272727
Precision: 0.5227441898933091
Sensitivity: 0.5227272727272727
Specifity: 0.5227272727272727
F1-Score: 0.5226385071604224
              precision    recall  f1-score   support

           0       0.52      0.54      0.53       110
           1       0.52      0.51      0.52       110

    accuracy                           0.52       220
   macro avg       0.52      0.52      0.52       220
weighted avg       0.52      0.52      0.52       220



In [14]:
max_values[0:8] #valores maximos de uma linha de um array

[0.7, 0.54, 0.62, 0.55, 0.6, 0.73, 0.52, 0.54]

In [19]:
aaa = pred_2[0]

In [25]:
aaa[1]

0.3

In [ ]:
#Otimizando Hiperparâmetros

In [45]:
param_dist = {"criterion": ['gini', 'entropy', 'log_loss'],
              "max_depth":[4,6,8,None],
              "max_features":['sqrt', 'log2', None]}

In [46]:
random_search = RandomizedSearchCV(estimator=model_1, param_distributions=param_dist,n_iter=40,cv=5)

In [47]:
random_search.fit(X,y)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=40,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [4, 6, 8, None],
                                        'max_features': ['sqrt', 'log2', None]})

In [48]:
random_search.best_estimator_

RandomForestClassifier(max_features='log2')

## 2. Empate

In [16]:
X = data_2.drop(columns=['result'])
y = data_2['result']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y,random_state=325)
X_test,X_val,y_test,y_val = train_test_split(X_test,y_test,test_size=0.5,stratify=y_test, random_state=3255)

In [17]:
X_train, y_train = rus.fit_resample(X_train, y_train)
X_test, y_test = rus.fit_resample(X_test, y_test)
X_val, y_val = rus.fit_resample(X_val, y_val)

In [18]:
model_2 = RandomForestClassifier()
model_2.fit(X_train,y_train)

RandomForestClassifier()

In [19]:
predictions_3 = model_2.predict(X_test)

In [20]:
print("Confusion Matrix")
print(confusion_matrix(y_test,predictions_3))
print("Accuracy:",accuracy_score(y_test,predictions_3))
print("Precision:",precision_score(y_test,predictions_3,average='weighted'))
print("Sensitivity:",recall_score(y_test,predictions_3,average='weighted')) #Sensitivity
print("Specifity:",recall_score(y_test,predictions_3,pos_label=0,average='weighted')) #Specifity
print("F1-Score:",f1_score(y_test,predictions_3,average='weighted'))

Confusion Matrix
[[67 42]
 [56 53]]
Accuracy: 0.5504587155963303
Precision: 0.5513050919982884
Sensitivity: 0.5504587155963303
Specifity: 0.5504587155963303
F1-Score: 0.5485970250169033


In [21]:
predictions_4 = model_2.predict(X_val)

In [22]:
print("Confusion Matrix")
print(confusion_matrix(y_val,predictions_4))
print("Accuracy:",accuracy_score(y_val,predictions_4))
print("Precision:",precision_score(y_val,predictions_4,average='weighted'))
print("Sensitivity:",recall_score(y_val,predictions_4,average='weighted')) #Sensitivity
print("Specifity:",recall_score(y_val,predictions_4,pos_label=0,average='weighted')) #Specifity
print("F1-Score:",f1_score(y_val,predictions_4,average='weighted'))

Confusion Matrix
[[62 48]
 [53 57]]
Accuracy: 0.5409090909090909
Precision: 0.5409937888198757
Sensitivity: 0.5409090909090909
Specifity: 0.5409090909090909
F1-Score: 0.540671834625323


In [23]:
prob_emp = model_2.predict_proba(X_val)

## 3. Derrota

In [24]:
X = data_3.drop(columns=['result'])
y = data_3['result']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y,random_state=325)
X_test,X_val,y_test,y_val = train_test_split(X_test,y_test,test_size=0.5,stratify=y_test, random_state=3255)

In [25]:
X_train, y_train = rus.fit_resample(X_train, y_train)
X_test, y_test = rus.fit_resample(X_test, y_test)
X_val, y_val = rus.fit_resample(X_val, y_val)

In [26]:
model_3 = RandomForestClassifier()
model_3.fit(X_train,y_train)

RandomForestClassifier()

In [27]:
predictions_5 = model_3.predict(X_test)

In [28]:
print("Confusion Matrix")
print(confusion_matrix(y_test,predictions_5))
print("Accuracy:",accuracy_score(y_test,predictions_5))
print("Precision:",precision_score(y_test,predictions_5,average='weighted'))
print("Sensitivity:",recall_score(y_test,predictions_5,average='weighted')) #Sensitivity
print("Specifity:",recall_score(y_test,predictions_5,pos_label=0,average='weighted')) #Specifity
print("F1-Score:",f1_score(y_test,predictions_5,average='weighted'))

Confusion Matrix
[[68 41]
 [62 47]]
Accuracy: 0.5275229357798165
Precision: 0.5285839160839161
Sensitivity: 0.5275229357798165
Specifity: 0.5275229357798165
F1-Score: 0.5230975086549285


In [29]:
predictions_6 = model_3.predict(X_val)

In [30]:
print("Confusion Matrix")
print(confusion_matrix(y_val,predictions_6))
print("Accuracy:",accuracy_score(y_val,predictions_6))
print("Precision:",precision_score(y_val,predictions_6,average='weighted'))
print("Sensitivity:",recall_score(y_val,predictions_6,average='weighted')) #Sensitivity
print("Specifity:",recall_score(y_val,predictions_6,pos_label=0,average='weighted')) #Specifity
print("F1-Score:",f1_score(y_val,predictions_6,average='weighted'))

Confusion Matrix
[[65 45]
 [61 49]]
Accuracy: 0.5181818181818182
Precision: 0.5185748058088484
Sensitivity: 0.5181818181818182
Specifity: 0.5181818181818182
F1-Score: 0.5156198072449318


In [40]:
prob_der = model_3.predict_proba(X_val)

### Exportando o Modelo

In [65]:
predictions_g2[0]

'D'

In [84]:
#verificar a ordem das classes
print(model_1.classes_)
print(model_2.classes_)
print(model_3.classes_)

['D' 'L' 'W']
['D' 'L' 'W']
['D' 'L' 'W']


In [70]:
#obter o dicionario de dados
resultados = {'V': prob_vit[0][1],
         'E': prob_emp[0][1],
         'D': prob_der[0][1]}

In [71]:
resultados

{'V': 0.3, 'E': 0.3, 'D': 0.37}

In [72]:
#obter o resultado de maior probabilidade
resultado_predito = max(resultados, key=resultados.get)

In [73]:
resultado_predito

'D'

In [74]:
dicionario = {'V': 0.2, 'E': 0.2, 'D': 0.1}

# Encontrar a chave com o maior valor
max(dicionario, key=dicionario.get)

'V'

In [31]:
import joblib
joblib.dump(model_1,'classificador-vit.joblib')
joblib.dump(model_2,'classificador-emp.joblib')
joblib.dump(model_3,'classificador-der.joblib')

['classificador-der.joblib']

In [33]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.1.post1


In [36]:
a = [0.56,0.44]

In [48]:
dados_testando = X[(X['p_shotsontarget'] == 0.3964) & (X['goals_shots'] == 0.09)]

In [49]:
dados_testando

,venue,points,gf,ga,possession,gdiff,shots,p_shotsontarget,goals_shots,penalty_goals,p_penaltyconverted,shotsontarget_against,p_saves,cleansheets,penalty_attempted_against,p_penalty_saved,cardscore,foulsdiff,offsides,crosses,interceptions,tackles_won,own_goals,opp_points,opp_gf,opp_ga,opp_possession,opp_gdiff,opp_shots,opp_p_shotsontarget,opp_goals_shots,opp_penalty_goals,opp_p_penaltyconverted,opp_shotsontarget_against,opp_p_saves,opp_cleansheets,opp_penalty_attempted_against,opp_p_penalty_saved,opp_cardscore,opp_foulsdiff,opp_offsides,opp_crosses,opp_interceptions,opp_tackles_won,opp_own_goals
3,0,5.0,1.2,1.2,0.462,0.0,13.4,0.3964,0.09,2.0,0.4,4.2,0.8168,1.0,1.0,0.0,4.0,-0.6,1.4,23.2,8.8,10.4,0.0,7.0,1.0,1.4,0.52,-2.0,13.6,0.2792,0.082,0.0,0.0,4.6,0.7252,1.0,0.0,0.0,4.2,-0.6,1.8,23.0,8.6,7.8,0.0


In [51]:
p_vitoria = model_1.predict_proba(dados_testando)

In [56]:
p_vitoria[0][0]

0.87